In [3]:
import os
import shutil
import random
import pandas as pd
import sqlite3

#from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix

from PIL import Image
import Augmentor

from tuner import utils
#import crud
from tuner import net

from database import db, mongodb

Using TensorFlow backend.


In [4]:
imgnet = net.lenet1
dataset_name = 'brain'
batch_size = 32
epochs = 10

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='')
parser.add_argument('--net', '-n', type=str,
                    help='model file name')
parser.add_argument('--weights', '-w', type=str,
                    help='weights file name')
parser.add_argument('--data', '-d', type=str,
                    help='set_data file name')
parser.add_argument('--id', '-i', type=str, default=None,
                    help='')
parser.add_argument('--epochs', '-e', type=int, default=100,
                    help='n epochs')
parser.add_argument('--batch', '-b', type=int, default=32,
                    help='batch size')
parser.add_argument('--augment', '-a', type=str, default=None,
                    help='brain or eyes')
args = parser.parse_args()

imgnet = eval(args.net)
dataset_name = args.data
batch_size = args.batch
epochs = args.epochs

In [3]:
dbname = 'curontab.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()
c.execute('PRAGMA foreign_keys=true')

In [4]:
client = pymongo.MongoClient('localhost',27017)
db = client.curontab
co = db.dataset

DBからデータセットの属性を取得

In [5]:
crud.datasets()

[{'data': 'data_5a25fdc9b037cba52f2e6b5f',
  'id': '5a25fdc9b037cba52f2e6b5f',
  'label2id': {'AB': 0, 'N': 1},
  'labels': ['AB', 'N'],
  'models': [],
  'name': 'eyes'},
 {'data': 'data_5a25fdc9b037cba52f2e6b60',
  'id': '5a25fdc9b037cba52f2e6b60',
  'label2id': {'MS': 3, 'N': 0, 'PD': 2, 'PS': 1},
  'labels': ['N', 'PS', 'PD', 'MS'],
  'models': [],
  'name': 'brain'}]

In [6]:
crud.models()

,id,name,validation_table,dataset_id


In [7]:
dataset= dataset_info = crud.dataset_by_name(dataset_name)
dataset_id = dataset['id']
labels = dataset['labels']
labels.sort()
label2id = dataset['label2id']
id2label = {i:label for label,i in label2id.items()}
df = data_df = crud.data_df_by_dataset_name(dataset_name)

ROOT_DIR = '.'
dataset_dir = os.path.join(ROOT_DIR, 'dataset', dataset['name'])

In [8]:
df_train, df_test = utils.train_test_split_df(df)

In [9]:
if dataset_name == 'eyes':
    df_train = utils.oversampling_df(df_train, 600)
x_train, y_train = utils.data_fromdf(df_train, dataset_dir, label2id, resize=(96,96))
x_test, y_test = utils.data_fromdf(df_test, dataset_dir, label2id, resize=(96,96))

In [10]:
p = Augmentor.Pipeline()

if dataset_name == 'eyes':
    p.shear(probability=0.2, max_shear_left=2, max_shear_right=2)
    p.crop_random(probability=0.2, percentage_area=0.95)
    p.resize(probability=1, width=96, height=96)
if dataset_name == 'brain':
    p.shear(probability=0.2, max_shear_left=2, max_shear_right=2)
    p.crop_random(probability=0.2, percentage_area=0.95)
    p.resize(probability=1, width=96, height=96)

g = p.keras_generator_from_array(x_train, y_train, batch_size=batch_size)
g = ( (xs/255,ys) for (xs,ys) in g)

In [11]:
import keras
n_out = len(labels)
model = net.lenet1(n_out)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

In [12]:
model.fit_generator(
    g,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    )

Epoch 1/10
7/7 [==============================] - 3s 484ms/step - loss: 1.7619 - acc: 0.2589 - val_loss: 10.3761 - val_acc: 0.2500
Epoch 2/10
7/7 [==============================] - 0s 33ms/step - loss: 1.3802 - acc: 0.3571 - val_loss: 6.8872 - val_acc: 0.5000
Epoch 3/10
7/7 [==============================] - 0s 33ms/step - loss: 1.3384 - acc: 0.4554 - val_loss: 9.1019 - val_acc: 0.3500
Epoch 4/10
7/7 [==============================] - 0s 33ms/step - loss: 1.2917 - acc: 0.3795 - val_loss: 9.9095 - val_acc: 0.3500
Epoch 5/10
7/7 [==============================] - 0s 33ms/step - loss: 1.1114 - acc: 0.5491 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/10
7/7 [==============================] - 0s 33ms/step - loss: 1.0021 - acc: 0.6071 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
7/7 [==============================] - 0s 33ms/step - loss: 1.0907 - acc: 0.5045 - val_loss: 4.8499 - val_acc: 0.7000
Epoch 8/10
7/7 [==============================] - 0s 33ms/step - loss: 1.0741 - acc: 0.5536 - v

In [13]:
df_preds = pd.DataFrame(model.predict(x_test))
f = pd.concat([df_preds,df_test], axis=1)
f = f.rename(columns={'index':'data_id'})
f['true'] = [ label2id[label] for label in f['label']]
f['pred'] = y_pred = model.predict(x_test).argmax(axis=1)
f['pred_label'] = [id2label[e] for e in f['pred']]
f['match'] = f['true'] == f['pred']
f['t_or_v'] = 'validation'

f

,0,1,2,3,data_id,fname,label,path,true,pred,pred_label,match,t_or_v
0,0.0,0.000000e+00,1.0,0.0,227,PS71.jpg,PS,./dataset/brain/PS/PS71.jpg,1,2,PD,False,validation
1,0.0,5.560023e-31,1.0,0.0,212,PS73.jpg,PS,./dataset/brain/PS/PS73.jpg,1,2,PD,False,validation
2,0.0,0.000000e+00,1.0,0.0,225,PS102.jpg,PS,./dataset/brain/PS/PS102.jpg,1,2,PD,False,validation
3,0.0,0.000000e+00,1.0,0.0,202,PS72.jpg,PS,./dataset/brain/PS/PS72.jpg,1,2,PD,False,validation
4,0.0,3.850853e-27,1.0,0.0,234,PS019.jpg,PS,./dataset/brain/PS/PS019.jpg,1,2,PD,False,validation
5,1.0,0.000000e+00,0.0,0.0,7,MS162.jpg,MS,./dataset/brain/MS/MS162.jpg,3,0,N,False,validation
6,1.0,0.000000e+00,0.0,0.0,48,MS142.jpg,MS,./dataset/brain/MS/MS142.jpg,3,0,N,False,validation
7,1.0,0.000000e+00,0.0,0.0,30,MS128.jpg,MS,./dataset/brain/MS/MS128.jpg,3,0,N,False,validation
8,1.0,0.000000e+00,0.0,0.0,3,MS161.jpg,MS,./dataset/brain/MS/MS161.jpg,3,0,N,False,validation
9,1.0,0.000000e+00,0.0,0.0,44,MS126.jpg,MS,./dataset/brain/MS/MS126.jpg,3,0,N,False,validation


In [14]:
from bson.objectid import ObjectId
validation_table = 'validation_{}'.format(str(ObjectId()))
f.to_sql(validation_table, conn)

In [15]:
model_name = '{}_{}'.format(dataset_name, imgnet.__name__)
q = '''
insert into model(
    name,
    validation_table,
    dataset_id,
    dataset_name
) values(?,?,?,?)
'''
v = (model_name, validation_table, dataset_id, dataset_name)
c.execute(q,v)

OperationalError: table model has no column named dataset_name

In [28]:
model.save('models/{}.hdf5'.format(model_name))

In [29]:
model_id = c.lastrowid
co = db.dataset
co.update({'name':dataset_name},{'$push':{
    'models':model_id}})

/home/fytroo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.


{'n': 1, 'nModified': 1, 'ok': 1, 'updatedExisting': True}

In [30]:
conn.commit()
conn.close()